# Initialize System

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -------------------------------
# Initialization Functions
# -------------------------------

def defect_phase(X, Y, x0, y0, charge, omega_R, charge_phase, R_phase):
    """
    Create a phase factor corresponding to a vortex‐like defect.
    """
    Xr = X - x0
    Yr = Y - y0
    R_local = np.sqrt(Xr**2 + Yr**2)
    Theta_local = np.arctan2(Yr, Xr)
    return np.exp(1j * (charge * Theta_local + charge_phase + 2 * np.pi * omega_R * R_local + R_phase))

def ring_defects(n_defects, r_ring, charge, omega_R, charge_phase=0, R_phase=0, offset=(0, 0)):
    """
    Generate a list of defect dictionaries arranged on a ring.
    """
    defects = []
    angles = np.linspace(0, 2*np.pi, n_defects, endpoint=False)
    for angle in angles:
        x0 = r_ring * np.cos(angle) + offset[0]
        y0 = r_ring * np.sin(angle) + offset[1]
        defects.append({
            "x0": x0, "y0": y0,
            "charge": charge,
            "omega_R": omega_R,
            "charge_phase": charge_phase,
            "R_phase": R_phase
        })
    return defects

def init_radial_annulus_vary_theta(X, Y, center_x, center_y, amplitude=0.1,
                                   radial_frequency=0.05, radial_decay=30,
                                   mode_radial=1, mode_theta=1, 
                                   phase_offset=0, theta_mod_amp=0.5):
    """
    Initialize a complex field with radially annular oscillations that are also modulated by theta.
    
    The field is given by:
    
      ψ(x,y) = amplitude · exp(–r / radial_decay) ·
                cos(2π·radial_frequency·r + phase_offset + theta_mod_amp · cos(mode_theta·θ)) ·
                exp(i · mode_radial·θ)
    
    where r = √((x – center_x)² + (y – center_y)²) and θ = arctan2(y – center_y, x – center_x).
    """
    r = np.sqrt((X - center_x)**2 + (Y - center_y)**2)
    theta = np.arctan2(Y - center_y, X - center_x)
    envelope = np.exp(-r / radial_decay)
    radial_oscillation = np.cos(2 * np.pi * radial_frequency * r +
                                phase_offset + theta_mod_amp * np.cos(mode_theta * theta))
    angular_phase = np.exp(1j * mode_radial * theta)
    return amplitude * envelope * radial_oscillation * angular_phase

# -------------------------------
# Grid Setup
# -------------------------------
Nx, Ny = 200, 200
L = 150  # domain half-length
x = np.linspace(-L, L, Nx)
y = np.linspace(-L, L, Ny)
X, Y = np.meshgrid(x, y)
R_grid = np.sqrt(X**2 + Y**2)
Theta = np.arctan2(Y, X)

# -------------------------------
# Field Initializations
# -------------------------------



# ψ₁ is initialized using a ring–defect method.
psi1 = np.ones((Nx, Ny), dtype=np.complex128)
# Create two rings of defects with opposite charge.
defects_ring1 = ring_defects(n_defects=2, r_ring=130, charge=6, omega_R=0.002, offset=(0, 0))
defects_ring2 = ring_defects(n_defects=1, r_ring=48, charge=-2, omega_R=0.002, offset=(0, 0))
for d in defects_ring1 + defects_ring2:
    psi1 *= defect_phase(X, Y, **d)
# Apply a tanh envelope to smooth the defect cores.
# and add a phase offset
psi1 *= np.tanh(R_grid / 50.0)*np.exp(1j * 0)


psi2 = np.ones((Nx, Ny), dtype=np.complex128)
# Create two rings of defects with opposite charge.
defects_ring1 = ring_defects(n_defects=2, r_ring=130, charge=6, omega_R=0.003, offset=(0, 0))
defects_ring2 = ring_defects(n_defects=4, r_ring=48, charge=-2, omega_R=0.003, offset=(0, 0))
for d in defects_ring1 + defects_ring2:
    psi2 *= defect_phase(X, Y, **d)
# Apply a tanh envelope to smooth the defect cores.
psi2 *= np.tanh(R_grid / 50.0)*np.exp(1j * 2*np.pi/3)

psi3 = np.ones((Nx, Ny), dtype=np.complex128)
# Create two rings of defects with opposite charge.
defects_ring1 = ring_defects(n_defects=2, r_ring=130, charge=6, omega_R=0.005, offset=(0, 0))
defects_ring2 = ring_defects(n_defects=6, r_ring=48, charge=-2, omega_R=0.005, offset=(0, 0))
for d in defects_ring1 + defects_ring2:
    psi3 *= defect_phase(X, Y, **d)
# Apply a tanh envelope to smooth the defect cores.
psi3 *= np.tanh(R_grid / 50.0)*np.exp(1j * 4*np.pi/3)

# OPTION #2
# ψ₂ is initialized using a radially annular waveform with theta variation.
# psi1 = init_radial_annulus_vary_theta(
#     X, Y, center_x=0, center_y=0,
#     amplitude=0.15, radial_frequency=0.006, radial_decay=30,
#     mode_radial=2, mode_theta=2, phase_offset=0, theta_mod_amp=0.5
# )

# psi2 = init_radial_annulus_vary_theta(
#     X, Y, center_x=0, center_y=0,
#     amplitude=0.15, radial_frequency=0.006, radial_decay=30,
#     mode_radial=2, mode_theta=3, phase_offset=np.pi/4, theta_mod_amp=0.5
# )

# # ψ₃ is initialized similarly with slightly different parameters.
# psi3 = init_radial_annulus_vary_theta(
#     X, Y, center_x=0, center_y=0,
#     amplitude=0.15, radial_frequency=0.006, radial_decay=30,
#     mode_radial=2, mode_theta=5, phase_offset=np.pi/2, theta_mod_amp=0.5
# )

# # H is also initialized with a radially annular pattern.
# H = init_radial_annulus_vary_theta(
#     X, Y, center_x=0, center_y=0,
#     amplitude=0.1, radial_frequency=0.04, radial_decay=40,
#     mode_radial=1, mode_theta=2, phase_offset=0, theta_mod_amp=0.4
# )

# -------------------------------
# initialize psi1-3 to complex valued waveforms with a phase offset
# -------------------------------
# psi1 = np.ones((Nx, Ny), dtype=np.complex128)
# psi2 = np.ones((Nx, Ny), dtype=np.complex128)
# psi3 = np.ones((Nx, Ny), dtype=np.complex128)


# omegas = [0.002, 0.004, 0.008]
# charges = [2, 2, 2]
# for om, ch in zip(omegas, charges):
#     psi1 *= defect_phase(X, Y, x0=0, y0=0, charge=ch, omega_R=om, charge_phase=0, R_phase=0)
#     psi1 *= np.cos(om*R_grid)


# omegas = [0.003, 0.006, 0.012]
# charges = [2, 2, 2]
# for om, ch in zip(omegas, charges):
#     psi2 *= defect_phase(X, Y, x0=0, y0=0, charge=ch, omega_R=om, charge_phase=2*np.pi/3, R_phase=0)
#     psi2 *= np.cos(om*R_grid)


# omegas = [0.005, 0.01, 0.02]
# charges = [2, 2, 2]
# for om, ch in zip(omegas, charges):
#     psi3 *= defect_phase(X, Y, x0=0, y0=0, charge=ch, omega_R=om, charge_phase=4*np.pi/3, R_phase=0)
#     psi3 *= np.cos(om*R_grid)


H = np.ones((Nx, Ny), dtype=np.complex128)

# φ (conformal factor) is initialized flat.
# phi = np.zeros((Nx, Ny))

# conformal factor φ is initialized to spatially varying values.
phi = np.zeros((Nx, Ny), dtype=np.float64)
phi = np.tanh(R_grid / 50.0) #* np.cos(2 * np.pi * R_grid / 50.0)

# Connectivity field, conn, is initialized uniformly (a small constant).
conn = np.ones((Nx, Ny), dtype=np.complex128) * 0.1

# Displacement fields U and V (in-plane offsets) are initialized at zero.
U = np.zeros((Nx, Ny))
V = np.zeros((Nx, Ny))

# -------------------------------
# Visualization
# -------------------------------
def plot_complex_field(ax, field, title):
    """
    Plot the amplitude of a complex field.
    """
    im = ax.imshow(np.abs(field), extent=[-L, L, -L, L], cmap='viridis', origin='lower')
    ax.set_title(f"{title} (Amplitude)")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

def plot_phase_field(ax, field, title):
    """
    Plot the phase of a complex field.
    """
    im = ax.imshow(np.angle(field), extent=[-L, L, -L, L], cmap='hsv', origin='lower')
    ax.set_title(f"{title} (Phase)")
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# For the real fields φ, U, and V, we just plot their values.
def plot_real_field(ax, field, title):
    im = ax.imshow(field, extent=[-L, L, -L, L], cmap='plasma', origin='lower')
    ax.set_title(title)
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

# Create subplots to visualize all fields.
fig, axs = plt.subplots(4, 2, figsize=(12, 18))

# ψ₁ Visualization
plot_complex_field(axs[0, 0], psi1, "ψ₁")
plot_phase_field(axs[0, 1], psi1, "ψ₁")

# ψ₂ Visualization
plot_complex_field(axs[1, 0], psi2, "ψ₂")
plot_phase_field(axs[1, 1], psi2, "ψ₂")

# ψ₃ Visualization
plot_complex_field(axs[2, 0], psi3, "ψ₃")
plot_phase_field(axs[2, 1], psi3, "ψ₃")

# H Visualization
plot_complex_field(axs[3, 0], H, "H")
plot_phase_field(axs[3, 1], H, "H")

plt.tight_layout()
plt.show()

# Create an additional figure for the real fields: φ, conn (amplitude), U, and V.
fig2, axs2 = plt.subplots(2, 2, figsize=(10, 8))

plot_real_field(axs2[0, 0], phi, "φ (Conformal Factor)")
plot_complex_field(axs2[0, 1], conn, "conn (Connectivity) - Amplitude")
plot_real_field(axs2[1, 0], U, "U (X Displacement)")
plot_real_field(axs2[1, 1], V, "V (Y Displacement)")

plt.tight_layout()
plt.show()


# Run Solver and Visualize Growth Process in Real-Time

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

# -------------------------------
# Helper Functions (unchanged parts omitted)
# -------------------------------
def norm(A):
    amin = A.min()
    amax = A.max()
    return ((A - amin) / (amax - amin + 1e-8)) * 255

def laplacian_2d(Z, dx, dy, boundary_cond='neumann', boundary_value=0.0):
    if boundary_cond == 'periodic':
        Z_left   = np.roll(Z, 1, axis=0)
        Z_right  = np.roll(Z, -1, axis=0)
        Z_up     = np.roll(Z, 1, axis=1)
        Z_down   = np.roll(Z, -1, axis=1)
    elif boundary_cond == 'neumann':
        Z_left  = np.zeros_like(Z)
        Z_right = np.zeros_like(Z)
        Z_up    = np.zeros_like(Z)
        Z_down  = np.zeros_like(Z)
        Z_left[1:,:]   = Z[:-1,:]
        Z_right[:-1,:] = Z[1:,:]
        Z_up[:,1:]     = Z[:,:-1]
        Z_down[:,:-1]  = Z[:,1:]
        Z_left[0,:]    = Z[0,:]
        Z_right[-1,:]  = Z[-1,:]
        Z_up[:,0]      = Z[:,0]
        Z_down[:,-1]   = Z[:,-1]
    else:
        raise ValueError(f"Unknown boundary condition: {boundary_cond}")
    return ((Z_left + Z_right - 2*Z) / (dx*dx) +
            (Z_up + Z_down - 2*Z) / (dy*dy))

def laplacian_2d_conformal(Z, dx, dy, phi, boundary_cond='neumann', boundary_value=0.0):
    flat_lap = laplacian_2d(Z, dx, dy, boundary_cond, boundary_value)
    return np.exp(-2 * phi) * flat_lap

def update_phi(phi, psi, dx, dy, dt, Gamma, kappa_phi, sigma, eta, bc='neumann'):
    psi_mag_sq = np.abs(psi)**2
    lap_phi    = laplacian_2d(phi, dx, dy, boundary_cond=bc)
    biharm_phi = laplacian_2d(laplacian_2d(phi, dx, dy, bc), dx, dy, bc)
    lap_psi    = laplacian_2d(psi_mag_sq, dx, dy, boundary_cond=bc)
    phi_update = -Gamma * (kappa_phi * biharm_phi - sigma * lap_phi + eta * lap_psi)
    return phi + dt * phi_update

def compute_bending_energy(X_embedded, Y_embedded, Z_embedded, dx, dy):
    # Compute gradients in x and y directions
    dX_dx = np.gradient(X_embedded, dx, axis=0)
    dY_dx = np.gradient(Y_embedded, dx, axis=0)
    dZ_dx = np.gradient(Z_embedded, dx, axis=0)
    dX_dy = np.gradient(X_embedded, dy, axis=1)
    dY_dy = np.gradient(Y_embedded, dy, axis=1)
    dZ_dy = np.gradient(Z_embedded, dy, axis=1)
    T_x = np.stack((dX_dx, dY_dx, dZ_dx), axis=-1)
    T_y = np.stack((dX_dy, dY_dy, dZ_dy), axis=-1)
    norm_T_x = np.linalg.norm(T_x, axis=-1, keepdims=True) + 1e-8
    norm_T_y = np.linalg.norm(T_y, axis=-1, keepdims=True) + 1e-8
    T_x_unit = T_x / norm_T_x
    T_y_unit = T_y / norm_T_y
    dT_x = np.diff(T_x_unit, axis=0)
    dT_y = np.diff(T_y_unit, axis=1)
    E_x = np.sum(dT_x**2, axis=-1)
    E_y = np.sum(dT_y**2, axis=-1)
    E_x = np.pad(E_x, ((0,1),(0,0)), mode='edge')
    E_y = np.pad(E_y, ((0,0),(0,1)), mode='edge')
    return E_x + E_y

def cgle_rhs_three_conformal_with_helfrich(psi1, psi2, psi3, H, dx, dy, phi,
                                           alpha_field, beta_map, gamma_map,
                                           kappa, kappa_H, d12=1, d13=1, d23=1,
                                           boundary_cond='neumann'):
    # Compute laplacians of ψ fields and H with conformal factor
    lap1 = laplacian_2d_conformal(psi1, dx, dy, phi, boundary_cond)
    lap2 = laplacian_2d_conformal(psi2, dx, dy, phi, boundary_cond)
    lap3 = laplacian_2d_conformal(psi3, dx, dy, phi, boundary_cond)
    lapH = laplacian_2d_conformal(H, dx, dy, phi, boundary_cond)
    
    intrinsic1 = alpha_field * psi1 - (1 + 1j*gamma_map[0]) * (np.abs(psi1)**2) * psi1 + (1 + 1j*beta_map[0]) * lap1
    intrinsic2 = alpha_field * psi2 - (1 + 1j*gamma_map[1]) * (np.abs(psi2)**2) * psi2 + (1 + 1j*beta_map[1]) * lap2
    intrinsic3 = alpha_field * psi3 - (1 + 1j*gamma_map[2]) * (np.abs(psi3)**2) * psi3 + (1 + 1j*beta_map[2]) * lap3
    
    # The following fixed phase shifts impose a 3-fold symmetry. You may make these adaptive later.
    coupling1 = kappa[0] * (np.exp(-1j * 2*np.pi/3)*psi2/(d12**2) +
                            np.exp(-1j * 4*np.pi/3)*psi3/(d13**2) -
                            psi1*(1/(d12**2) + 1/(d13**2)))
    coupling2 = kappa[1] * (np.exp(-1j * 2*np.pi/3)*psi3/(d23**2) +
                            np.exp(-1j * 4*np.pi/3)*psi1/(d12**2) -
                            psi2*(1/(d23**2) + 1/(d12**2)))
    coupling3 = kappa[2] * (np.exp(-1j * 2*np.pi/3)*psi1/(d13**2) +
                            np.exp(-1j * 4*np.pi/3)*psi2/(d23**2) -
                            psi3*(1/(d13**2) + 1/(d23**2)))
    
    helfrich_coupling1 = kappa_H * (np.exp(-1j * 2*np.pi/3)*H/(d12**2) +
                                    np.exp(-1j * 4*np.pi/3)*H/(d13**2) -
                                    psi1*(1/(d12**2) + 1/(d13**2)))
    helfrich_coupling2 = kappa_H * (np.exp(-1j * 2*np.pi/3)*H/(d23**2) +
                                    np.exp(-1j * 4*np.pi/3)*H/(d12**2) -
                                    psi2*(1/(d23**2) + 1/(d12**2)))
    helfrich_coupling3 = kappa_H * (np.exp(-1j * 2*np.pi/3)*H/(d13**2) +
                                    np.exp(-1j * 4*np.pi/3)*H/(d23**2) -
                                    psi3*(1/(d13**2) + 1/(d23**2)))
    
    rhs1 = intrinsic1 + coupling1 + helfrich_coupling1
    rhs2 = intrinsic2 + coupling2 + helfrich_coupling2
    rhs3 = intrinsic3 + coupling3 + helfrich_coupling3
    
    intrinsic_H = alpha_field * H - (1 + 1j*gamma_map[0]) * (np.abs(H)**2) * H + (1 + 1j*beta_map[0]) * lapH
    coupling_H = kappa_H * (np.exp(-1j * 2*np.pi/3)*psi1/(d12**2) +
                            np.exp(-1j * 4*np.pi/3)*psi2/(d13**2) +
                            np.exp(-1j * 6*np.pi/3)*psi3/(d23**2) -
                            H*(1/(d12**2)+1/(d13**2)+1/(d23**2)))
    rhs_H = intrinsic_H + coupling_H
    
    return rhs1, rhs2, rhs3, rhs_H

# -------------------------------
# New: Connectivity Field Dynamics (Modified)
# -------------------------------
def update_connectivity(conn, E_bend, dx, dy, dt, alpha_conn, beta_conn, gamma_conn, E_thresh, D_conn, delta_conn=1.0, zeta_conn=0.1):
    """
    Evolve the connectivity field (conn) with a CGLE–like dynamics.
    The effective growth rate is modulated smoothly by the local bending energy.
      - For E_bend > E_thresh, connectivity dissolves.
      - For E_bend < E_thresh, connectivity is promoted.
    A logistic function is used for a smooth transition, and a quartic damping term prevents runaway growth.
    """
    # Smooth logistic modulation
    alpha_eff = alpha_conn / (1 + np.exp((E_bend - E_thresh)/delta_conn))
    lap_conn = laplacian_2d(conn, dx, dy, boundary_cond='neumann')
    # Add a quartic damping term for stabilization
    dconn_dt = D_conn * lap_conn + alpha_eff * conn - (1 + 1j*gamma_conn) * (np.abs(conn)**2) * conn \
               + (1 + 1j*beta_conn) * lap_conn - zeta_conn * (np.abs(conn)**4) * conn
    return conn + dt * dconn_dt

# -------------------------------
# Simulation Setup (grid, initializations, parameters)
# -------------------------------
# Define spatial grid (for example purposes; define x, y, Nx, Ny appropriately)
Nx, Ny = 200, 200
L = 150  # half-length of the domain
x = np.linspace(-L, L, Nx)
y = np.linspace(-L, L, Ny)
X, Y = np.meshgrid(x, y)
dx, dy = x[1]-x[0], y[1]-y[0]

# Initialize fields: psi1, psi2, psi3, H, phi, connectivity (conn) and displacement fields U, V.
# (Here we use simple random initializations; you may need to adjust these based on your problem.)
# psi1 = (np.random.rand(Nx, Ny) + 1j*np.random.rand(Nx, Ny)) * 0.1
# psi2 = (np.random.rand(Nx, Ny) + 1j*np.random.rand(Nx, Ny)) * 0.1
# psi3 = (np.random.rand(Nx, Ny) + 1j*np.random.rand(Nx, Ny)) * 0.1
# H    = (np.random.rand(Nx, Ny) + 1j*np.random.rand(Nx, Ny)) * 0.1
# phi  = np.zeros((Nx, Ny))
# conn = (np.random.rand(Nx, Ny) + 1j*np.random.rand(Nx, Ny)) * 0.1
# U = np.zeros((Nx, Ny))
# V = np.zeros((Nx, Ny))

# Simulation parameters
Nt = 700        # Number of time steps
dt = 0.008       # Time step size
alpha_base = 4.0
beta_map  = 0.9 * np.ones(3)
gamma_map = -1.05 * np.ones(3)
kappa0 = np.array([0.2, 0.2, 0.2])
kappa = np.zeros((3, Nx, Ny))
kappa[0] = kappa0[0]
kappa[1] = kappa0[1]
kappa[2] = kappa0[2]
Gamma     = 1e-1
kappa_phi = 2e-3
sigma     = 1e-3
eta       = 2e-1
k_b       = 0.1
C_height  = 50

w_growth    = 0.6 # Growth rate for the conformal factor
w_curvature = 0.4
w_adhesion  = 0.5

D_normal = 0.5
kappa_H = 0.2

# Connectivity parameters
alpha_conn = 4.0
beta_conn  = 0.1
gamma_conn = -1.05
D_conn     = 0.1

E_thresh   = 5.5  # Bending energy threshold

delta_conn = 1.0  # Logistic transition width for connectivity modulation
zeta_conn  = 0.1  # Quartic damping coefficient for connectivity

# -----------------------------------
# (Optional) Visualization Setup (OpenCV)
# -----------------------------------
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps_video = 20.0
frame_size = (2 * Ny, Nx) # Adjusted for video output
video_writer = cv2.VideoWriter("three_psi_H_conn_RK4.mp4", fourcc, fps_video, frame_size)
visualization_interval = 1

# -------------------------------
# Main Simulation Loop (RK4 Integration)
# -------------------------------
for n in tqdm(range(Nt)):
    bc = 'neumann'
    
    # Compute current 3D embedding
    E_x = X + U
    E_y = Y + V
    E_z = C_height * phi
    E = np.stack((E_x, E_y, E_z), axis=-1)
    
    # Compute local surface normals (finite difference approach)
    dE_dx = np.gradient(E, dx, axis=0)
    dE_dy = np.gradient(E, dy, axis=1)
    n_local = np.cross(dE_dx, dE_dy)
    norm_n = np.linalg.norm(n_local, axis=-1, keepdims=True) + 1e-8
    n_unit = n_local / norm_n  # Re-normalize to ensure unit length
    
    # Compute local bending energy using the embedded surface
    E_bend = compute_bending_energy(E_x, E_y, E_z, dx, dy)
    
    # Update connectivity field based on local bending energy.
    conn = update_connectivity(conn, E_bend, dx, dy, dt, alpha_conn, beta_conn, gamma_conn, E_thresh, D_conn, delta_conn, zeta_conn)
    
    # Use connectivity field to modulate spring stiffness:
    modulation_factor = np.real(conn) * np.abs(conn)**2 * 0.9 * (1 + np.tanh(phi))
    kappa[0] = kappa0[0] * modulation_factor
    kappa[1] = kappa0[1] * modulation_factor
    kappa[2] = kappa0[2] * modulation_factor
    kappa_H_eff = kappa_H * modulation_factor
    
    # Update alpha_field based on bending energy using a rational function instead of an exponential decay.
    alpha_field = alpha_base / (1 + k_b * E_bend)
    
    # RK4 integration for ψ and H fields
    k1_1, k1_2, k1_3, k1_H = cgle_rhs_three_conformal_with_helfrich(
        psi1, psi2, psi3, H, dx, dy, phi,
        alpha_field, beta_map, gamma_map, kappa, kappa_H_eff, boundary_cond=bc)
    k2_1, k2_2, k2_3, k2_H = cgle_rhs_three_conformal_with_helfrich(
        psi1 + 0.5*dt*k1_1, psi2 + 0.5*dt*k1_2, psi3 + 0.5*dt*k1_3, H + 0.5*dt*k1_H,
        dx, dy, phi, alpha_field, beta_map, gamma_map, kappa, kappa_H_eff, boundary_cond=bc)
    k3_1, k3_2, k3_3, k3_H = cgle_rhs_three_conformal_with_helfrich(
        psi1 + 0.5*dt*k2_1, psi2 + 0.5*dt*k2_2, psi3 + 0.5*dt*k2_3, H + 0.5*dt*k2_H,
        dx, dy, phi, alpha_field, beta_map, gamma_map, kappa, kappa_H_eff, boundary_cond=bc)
    k4_1, k4_2, k4_3, k4_H = cgle_rhs_three_conformal_with_helfrich(
        psi1 + dt*k3_1, psi2 + dt*k3_2, psi3 + dt*k3_3, H + dt*k3_H,
        dx, dy, phi, alpha_field, beta_map, gamma_map, kappa, kappa_H_eff, boundary_cond=bc)
    
    psi1 += (dt/6.0) * (k1_1 + 2*k2_1 + 2*k3_1 + k4_1)
    psi2 += (dt/6.0) * (k1_2 + 2*k2_2 + 2*k3_2 + k4_2)
    psi3 += (dt/6.0) * (k1_3 + 2*k2_3 + 2*k3_3 + k4_3)
    H    += (dt/6.0) * (k1_H + 2*k2_H + 2*k3_H + k4_H)
    
    # Update φ from contributions of the ψ fields.
    phi_input = (w_growth    * np.abs(psi1)**2 +
                 w_curvature * np.abs(psi2)**2 +
                 w_adhesion  * np.abs(psi3)**2)
    phi = update_phi(phi, phi_input, dx, dy, dt, Gamma, kappa_phi, sigma, eta, bc=bc)
    
    # Normal-oriented growth update.
    growth_amp = np.abs(psi1)
    growth_amp_norm = growth_amp / (growth_amp.max() + 1e-8)
    delta_E = dt * D_normal * growth_amp_norm[..., None] * n_unit
    E_new = E + delta_E
    U = E_new[..., 0] - X
    V = E_new[..., 1] - Y
    phi = E_new[..., 2] / C_height

    # Periodic smoothing of U and V for embedding regularization (every 500 steps)
    if n % 500 == 0:
        U = cv2.GaussianBlur(U.astype(np.float32), (5,5), 0)
        V = cv2.GaussianBlur(V.astype(np.float32), (5,5), 0)

    # (Optional) Visualization via OpenCV diagnostics.
    if n % visualization_interval == 0:
        lap1 = np.abs(laplacian_2d_conformal(psi1, dx, dy, phi, bc))
        lap2 = np.abs(laplacian_2d_conformal(psi2, dx, dy, phi, bc))
        lap3 = np.abs(laplacian_2d_conformal(psi3, dx, dy, phi, bc))
        red   = norm(lap1).astype(np.uint8)
        green = norm(lap2).astype(np.uint8)
        blue  = norm(lap3).astype(np.uint8)
        rgb_laplacian = cv2.merge([blue, green, red])
        red   = norm(np.abs(psi1)**2 * np.real(psi1)).astype(np.uint8)
        green = norm(np.abs(psi2)**2 * np.real(psi2)).astype(np.uint8)
        blue  = norm(np.abs(psi3)**2 * np.real(psi3)).astype(np.uint8)
        rgb_abs = cv2.merge([blue, green, red])
        
        tissue_contours = cv2.applyColorMap(norm(phi).astype(np.uint8), cv2.COLORMAP_VIRIDIS)
        
        # final_display = np.hstack((tissue_contours, rgb_abs, rgb_laplacian))

        # without the tissue contours
        final_display = np.hstack((rgb_abs, rgb_laplacian))

        cv2.imshow("Simulation Visualization", final_display)
        video_writer.write(final_display)
        if cv2.waitKey(1) == ord('q'):
            break

cv2.destroyAllWindows()
video_writer.release()

# -------------------------------
# Final 2D Projection Visualization
# -------------------------------
X_embedded = X + U
Y_embedded = Y + V
Z_embedded = C_height * phi

plt.figure(figsize=(8, 6))
plt.contourf(X_embedded, Y_embedded, Z_embedded, levels=100, cmap='viridis')
plt.xlabel("X")
plt.ylabel("Y")
plt.title("Final 2D Projection of Tissue Surface")
plt.colorbar(label="Height (Z)")
plt.show()
